# Import Libraries

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import special
from scipy import integrate

import openturns as ot
import time

from SALib.sample import morris as salib_morris
from SALib.analyze import morris as salib_analyze
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import matplotlib as mpl

# Data Generation

In [2]:
def generate_samples(min_value, max_value, num_samples):
    distribution = ot.ComposedDistribution([ot.Uniform(min_value, max_value)] * 1)
    importanceDistribution = ot.ComposedDistribution([ot.Uniform(min_value, max_value)] * 1)
    experiment = ot.ImportanceSamplingExperiment(distribution, importanceDistribution, num_samples)
    return np.array(experiment.generate()).reshape(-1)

# Set seed
seed = 3424
ot.RandomGenerator.SetSeed(seed)

samplesize = 10

# Generate samples for each variable
Co_t = generate_samples(1, 120, samplesize)
H_t = generate_samples(1, 16, samplesize)
W_t = generate_samples(1, 305, samplesize)
v_t = generate_samples(1, 60, samplesize)
alx_t = generate_samples(1.5, 30.5, samplesize)
aly_t = generate_samples(0.5, 10.2, samplesize)
alz_t = generate_samples(0.0075, 1.5, samplesize)
R_t = generate_samples(1, 2, samplesize)
SD_g_t = generate_samples(0, 1, samplesize)
Deg_C_t = generate_samples(0.1, 0.45, samplesize)

# Save to file
filename = f'Dataset_Seed_{seed}.txt'
np.savetxt(filename, np.column_stack((Co_t, H_t, W_t, v_t, alx_t, aly_t, alz_t, R_t, SD_g_t, Deg_C_t)))

# Transient Model

### Calculation Functions 

In [3]:
def C_transient(x,t):
        
        # Boundary Condition
        
        if x<=1e-6:
            if y <= W/2 and y >= -W/2 and z <= z_2 and z >= z_1:
                C=Co*np.exp(-ga*t)
            else:
                C=0
        else:
            a = Co*np.exp(-ga*t)*x/(8*np.sqrt(np.pi*Dxr))
            roots = sp.special.roots_legendre(m)[0]
            weights = sp.special.roots_legendre(m)[1]
    
            #scaling
        
            bot = 0
            top = np.sqrt(np.sqrt(t))
            Tau = (roots*(top-bot)+top+bot)/2
            Tau4 = Tau**4
    
            #calculation
                
            xTerm = (np.exp(-(((la-ga)*Tau4)+((x-vr*Tau4)**2)/(4*Dxr*Tau4))))/(Tau**3)
            yTerm = sp.special.erfc((y-W/2)/(2*np.sqrt(Dyr*Tau4))) - sp.special.erfc((y+W/2)/(2*np.sqrt(Dyr*Tau4)))
            zTerm = sp.special.erfc((z-z_2)/(2*np.sqrt(Dzr*Tau4))) - sp.special.erfc((z-z_1)/(2*np.sqrt(Dzr*Tau4)))
            Term = xTerm * yTerm * zTerm
            Integrand = Term*(weights*(top-bot)/2)
            C = a*4*sum(Integrand)
        return C
    
    #function for Lmax depending on time past
    
def Lmax(t):
        #x_array = np.array([0])                                                     #plotting
        #c_array = np.array([C_transient(0,t)])                                                #plotting
        x_min = 0  # minimum plume length to avoid unnecessary calculation (NOT a fixed value!)
        x_lower = x_min
        x_upper = 10000

        optimal_x = 0

        # Optimization Scheme 1
        while x_upper - x_lower > 1:
            #print(optimal_x)
            x_mid = (x_lower + x_upper) / 2
            C_mid = C_transient(x_mid, t)
            if C_mid >= Cthres:

                optimal_x = x_mid
                x_lower = x_mid
            else:
                x_upper = x_mid
        #print((C_transient(x_upper, t),Cthres,optimal_x,x_upper))
        if (optimal_x < x_upper) and (C_transient(x_upper, t) >= Cthres) :
            #print(C_transient(x_upper, t),Cthres)
            optimal_x = x_upper

        x = optimal_x
        return x

### Data Generation

In [4]:
lmax_transient = []
t_transient = []

for i in range(samplesize):
    ## Input Parameters
    Cthres = 5e-5               #[mg/l]                                             Input Box (lower limit: >0, upper limit: <Co)
    
    # time
    #t = 100                    #[y]                                                Input Box (Default = 20, if below 15, use: (1/la)+5,  upper limit 1000)
    
    # Geometry - centreline
    y = 0                       #[m]                                                fixed (no Input)
    z_1 = 0                     #[m]                                                fixed (no input)
    z_2 = H_t[i]                     #[m]                                               Input slider (lower limit: >0, upper limit:50)
    
    # Source term
    Co = Co_t[i]                   #[mg/l]                                              Input slider (lower limit: >0, upper limit:1000)
    z = (z_1+z_2)/2             #[m]                                                fixed (no Input)
    W = W_t[i]                      #[m]                                                Input slider (lower limit: >0, upper limit:1000)
    
    # hydraulic & mixing
    v = v_t[i]                      #[m/y]                                              Input Box (lower limit: 10, upper limit: 1000)
    al_x = alx_t[i]                   #[m]                                                Input slider (lower limit: 1, upper limit: 100, default: 10)
    al_y = aly_t[i]                    #[m]                                                Input Box (lower limit: 0.1, upper limit: 10, default: 0.5)
    al_z = alz_t[i]                 #[m]                                                Input Box (lower limit: 0.01, upper limit: 1, default: 0.05)
    Df = 0                      #[m^2/y]                                            Input Box (lower limit: 0 upper limit: 0.1, default: 0)
    
    # reaction terms
    R = R_t[i]                       #[-]                                                Input Box (lower limit: 1, upper limit:100, default: 1)
    ga = SD_g_t[i]                   #[1/y]                                              Input Box (lower limit: 0, upper limit: 1, default: 0)
    la = Deg_C_t[i]                 #[1/y]                                              Input slider (lower limit: 0, upper limit: 1, default: 0.1)
    
    # Gauss points: max 256
    m = 60                     #[-]                                                 Input Box (possible values: 4,5,6,10,15,20,60,104,256; default: 60)
    
    Dx = al_x*v + Df            #[m^2/y]
    Dy = al_y*v + Df            #[m^2/y]
    Dz = al_z*v + Df            #[m^2/y]
    
    # used data
    vr = v/R                    #[m/y]
    Dyr = Dy/R                  #[m^2/y]
    Dxr = Dx/R                  #[m^2/y]
    Dyr = Dy/R                  #[m^2/y]
    Dzr = Dz/R                  #[m^2/y]
    
    t_min = 0                                                                       #starting time for numerical evaluation to avoid unnecessary calculation (NOT a fixed value!)
    t = t_min
    delta_t = 1                                                                     #delta t for defining steady state
    Lmax1 = Lmax(t)                                                                 #must be big otherwise maximum of function Lmax(x) can stop the loop
    Lmax2 = Lmax(t+delta_t)
    Lmax_array = np.array([0])                                                      #array for plotting
    t_array = np.array([0])                                                         #array for plotting
    
    while Lmax2 > Lmax1:                                                            #delta Lmax > 0 -> Criterium for steady state
        t = t+1                                                                     #resolution of time steps
        Lmax1 = Lmax(t)
        Lmax2 = Lmax(t+delta_t)
        Lmax_array = np.append(Lmax_array, Lmax(t))                                 #appending data to array (plotting)
        t_array = np.append(t_array, t)                                             #appending data to array (plotting)
        
    lmax_transient.append(Lmax(t))
    t_transient.append(t)
filename = f'Lmax_transient_{seed}.txt' 
np.savetxt(filename, np.column_stack((lmax_transient, t_transient)))

/tmp/ipykernel_20225/4009115278.py:24: RuntimeWarning: divide by zero encountered in divide
  xTerm = (np.exp(-(((la-ga)*Tau4)+((x-vr*Tau4)**2)/(4*Dxr*Tau4))))/(Tau**3)
/tmp/ipykernel_20225/4009115278.py:24: RuntimeWarning: invalid value encountered in divide
  xTerm = (np.exp(-(((la-ga)*Tau4)+((x-vr*Tau4)**2)/(4*Dxr*Tau4))))/(Tau**3)
/tmp/ipykernel_20225/4009115278.py:25: RuntimeWarning: divide by zero encountered in divide
  yTerm = sp.special.erfc((y-W/2)/(2*np.sqrt(Dyr*Tau4))) - sp.special.erfc((y+W/2)/(2*np.sqrt(Dyr*Tau4)))
/tmp/ipykernel_20225/4009115278.py:26: RuntimeWarning: divide by zero encountered in divide
  zTerm = sp.special.erfc((z-z_2)/(2*np.sqrt(Dzr*Tau4))) - sp.special.erfc((z-z_1)/(2*np.sqrt(Dzr*Tau4)))


# Steady-State Model

In [6]:
lmax_steady = []
for i in range(len(Co_t)):
    #Input
    
    W = W_t[i]
    H = H_t[i]
    C_0 = Co_t[i]
    v = v_t[i]
    alpha_L = alx_t[i]
    alpha_Th = aly_t[i]
    alpha_Tv = alz_t[i]
    D = 0
    D_x = v*alpha_L+D
    D_y = v*alpha_Th+D
    D_z = v*alpha_Tv+D
    lambda_eff= Deg_C_t[i]
    Cthres = 5e-5
    
    #Function
    
    def C_steady(x,y,z):
        
        def C_ps(z,y):
            r = np.sqrt((lambda_eff+v**2/(4*D_x))*(x**2/D_x+y**2/D_y+z**2/D_z)) 
            return C_0*(2*H*W)/(2*np.pi*np.sqrt(D_x*D_y*D_z))*(lambda_eff+v**2/(4*D_x))**(3/2)*x/r**2*(1+(1/r))*np.exp((v*x)/(2*D_x)-r)
                  
        return 1/(2*H*W)*integrate.dblquad(C_ps, -W/2, W/2, -H, H)[0]
    
    x_min = 1  # minimum value to avoid unnecessary calculations (NOT a fixed value!)
    x_lower = x_min
    x_upper = 100000

    optimal_x = 0

    # Optimization Scheme 2
    while x_upper - x_lower > 1:
        x_mid = (x_lower + x_upper) // 2
        C_mid = C_steady(x_mid, 0, 0)
        if C_mid >= Cthres:
            optimal_x = x_mid
            x_lower = x_mid
        else:
            x_upper = x_mid

    if (optimal_x < x_upper) and (C_steady(x_upper, 0, 0) >= Cthres):
        optimal_x = x_upper

    lmax_steady.append(optimal_x)
# end_time2 = time.perf_counter()
# print(end_time2-end_time)
filename = f'Lmax_steady_{seed}.txt'
np.savetxt(filename, lmax_steady)

# Morris Method for Sensitivity Analysis

In [7]:
# Define your function for the arbitrary calculation
def calculate_output(x):
    Co_t, H_t, W_t, v_t, alx_t, aly_t, alz_t, R_t, SD_g_t, Deg_C_t = x
    lmax_transient = 0
    t_transient = 0
    ## Input Parameters
    Cthres = 5e-5               #[mg/l]
    # Geometry - centreline
    y = 0                       #[m]                                                fixed (no Input)
    z_1 = 0                     #[m]                                                fixed (no input)
    z_2 = H_t                     #[m]                                               Input slider (lower limit: >0, upper limit:50)
    # Source term
    Co = Co_t    
    z = (z_1+z_2)/2                     #[m]
    W = W_t                      #[m]                                                Input slider (lower limit: >0, upper limit:1000)
    
    # hydraulic & mixing
    v = v_t                      #[m/y]                                              Input Box (lower limit: 10, upper limit: 1000)
    al_x = alx_t                   #[m]                                                Input slider (lower limit: 1, upper limit: 100, default: 10)
    al_y = aly_t                    #[m]                                                Input Box (lower limit: 0.1, upper limit: 10, default: 0.5)
    al_z = alz_t                 #[m]                                                Input Box (lower limit: 0.01, upper limit: 1, default: 0.05)
    Df = 0                      #[m^2/y]                                            Input Box (lower limit: 0 upper limit: 0.1, default: 0)
    
    # reaction terms
    R = R_t                       #[-]                                                Input Box (lower limit: 1, upper limit:100, default: 1)
    ga = SD_g_t                   #[1/y]                                              Input Box (lower limit: 0, upper limit: 1, default: 0)
    la = Deg_C_t                 #[1/y]                                              Input slider (lower limit: 0, upper limit: 1, default: 0.1)
    
    # Gauss points: max 256
    m = 256                     #[-]                                                 Input Box (possible values: 4,5,6,10,15,20,60,104,256; default: 60)
    
    Dx = al_x*v + Df            #[m^2/y]
    Dy = al_y*v + Df            #[m^2/y]
    Dz = al_z*v + Df            #[m^2/y]
    
    # used data
    vr = v/R                    #[m/y]
    Dyr = Dy/R                  #[m^2/y]
    Dxr = Dx/R                  #[m^2/y]
    Dyr = Dy/R                  #[m^2/y]
    Dzr = Dz/R                  #[m^2/y]
    
    
    t_min = 0                                                                       #starting time for numerical evaluation to avoid unnecessary calculation (NOT a fixed value!)
    t = t_min
    delta_t = 1                                                                     #delta t for defining steady state
    Lmax1 = Lmax(t)                                                                 #must be big otherwise maximum of function Lmax(x) can stop the loop
    Lmax2 = Lmax(t+delta_t)
    Lmax_array = np.array([0])                                                      #array for plotting
    t_array = np.array([0])                                                         #array for plotting
    
    while Lmax2 > Lmax1:                                                            #delta Lmax > 0 -> Criterium for steady state
        t = t+1                                                                     #resolution of time steps
        Lmax1 = Lmax(t)
        Lmax2 = Lmax(t+delta_t)
        Lmax_array = np.append(Lmax_array, Lmax(t))                                 #appending data to array (plotting)
        t_array = np.append(t_array, t)                                             #appending data to array (plotting)
        
    lmax_transient=(Lmax(t))
    t_transient=(t)

    #print(lmax_transient)
    #output = x1 + 2 * x2 + 0.5 * x3**2 + np.sin(x4) + np.exp(-x5)
    return lmax_transient

# Define the bounds for each variable
problem_bounds = [
    [1, 120],       # Variable Co_t bounds
    [1, 16],       # Variable H_t bounds
    [1,305],      # Variable W_t bounds
    [1,60],   # Variable v_t bounds
    [1.5,30.5],        # Variable alx_t bounds
    [0.5,10.2], #variable aly_t bounds
    [0.0075,1.5],         #variable alz_t bounds
    [1,2],              #variable R_t bounds
    [0,1],              #variable SD_g bounds
    [0.1,0.45]              #variable Deg_C bounds
]

# Set the number of trajectories and sample size
r = 10
sample_size = 1000

# Perform the Morris sensitivity analysis
problem = {
    'num_vars': len(problem_bounds),
    'names': ["Co_t"," H_t"," W_t"," v_t", "alx_t"," aly_t"," alz_t"," R_t"," SD_g_t", "Deg_C_t"],
    'bounds': problem_bounds,
}

# Generate the sample using SALib
sample = salib_morris.sample(problem, sample_size, num_levels=r)

# Evaluate the function at the sample points
Y = []
for i, x in enumerate(sample):
    Y.append(calculate_output(x))
    progress = (i + 1) / len(sample) * 100
    print(f"Progress: {progress:.2f}%")


# Convert Y to a NumPy array
Y = np.array(Y)

# Perform the Morris sensitivity analysis using SALib
sensitivity_analysis = salib_analyze.analyze(problem, sample, Y, conf_level=0.95, num_levels=r)

mu_array = []
sigma_array = []

# Print the elementary effects
for i in range(len(problem_bounds)):
    mu_array.append(sensitivity_analysis['mu_star'][i])
    sigma_array.append(sensitivity_analysis['sigma'][i])

results = np.column_stack((mu_array,sigma_array))
df = pd.DataFrame(results, columns=['mu_star', 'sigma'])
df['Parameter'] = problem['names']
df = df.set_index('Parameter')
df.to_csv('results_lmax', sep=' ', index=True, header=True)

/tmp/ipykernel_20225/4009115278.py:24: RuntimeWarning: divide by zero encountered in divide
  xTerm = (np.exp(-(((la-ga)*Tau4)+((x-vr*Tau4)**2)/(4*Dxr*Tau4))))/(Tau**3)
/tmp/ipykernel_20225/4009115278.py:24: RuntimeWarning: invalid value encountered in divide
  xTerm = (np.exp(-(((la-ga)*Tau4)+((x-vr*Tau4)**2)/(4*Dxr*Tau4))))/(Tau**3)
/tmp/ipykernel_20225/4009115278.py:25: RuntimeWarning: divide by zero encountered in divide
  yTerm = sp.special.erfc((y-W/2)/(2*np.sqrt(Dyr*Tau4))) - sp.special.erfc((y+W/2)/(2*np.sqrt(Dyr*Tau4)))
/tmp/ipykernel_20225/4009115278.py:26: RuntimeWarning: divide by zero encountered in divide
  zTerm = sp.special.erfc((z-z_2)/(2*np.sqrt(Dzr*Tau4))) - sp.special.erfc((z-z_1)/(2*np.sqrt(Dzr*Tau4)))


Progress: 0.01%
Progress: 0.02%
Progress: 0.03%
Progress: 0.04%
Progress: 0.05%
Progress: 0.05%
Progress: 0.06%
Progress: 0.07%
Progress: 0.08%
Progress: 0.09%
Progress: 0.10%
Progress: 0.11%
Progress: 0.12%
Progress: 0.13%
Progress: 0.14%
Progress: 0.15%
Progress: 0.15%
Progress: 0.16%
Progress: 0.17%


KeyboardInterrupt: 

# Polynomial Model

In [ ]:
#%% Training Data

data = pd.read_csv('Dataset_Seed_3424.txt', header = None, names = ['C0','H','W','v','alx','aly','alz','R','Gamma','Lambda'], sep=' ')
plumeresults = pd.read_csv('Lmax_transient_Seed_3424.txt', header = None, names = ['Lmax', 'T_Lmax'], sep = ' ')

gamma = data['Gamma']
gamma_2 = gamma**2
lambda_eff = data['Lambda']
lambda_eff_2 = lambda_eff**2
lambda_eff_gamma = lambda_eff*gamma
v = data['v']
v_2 = v**2
lambda_eff_v = lambda_eff*v

lmax = plumeresults['Lmax']
t_lmax = plumeresults['T_Lmax']

ones = np.ones(len(gamma))

Params_T = np.column_stack((ones, lambda_eff, gamma, lambda_eff_2, gamma_2, lambda_eff_gamma))
Params_L = np.column_stack((ones, lambda_eff, v, lambda_eff_2, v_2, lambda_eff_v))

Polynomial_coeff_L = np.linalg.lstsq(Params_L,lmax,rcond=None)[0]
Polynomial_coeff_T = np.linalg.lstsq(Params_T,t_lmax,rcond=None)[0]
print(Polynomial_coeff_L, Polynomial_coeff_T)